In [ ]:
#################### I apologize that this notebook is only 15% of the way done. It was mainly a POC before my Masters.
#################### It started early, so attached is a picture of what was going to be analyzed and correlated.
#################### Have a great day!

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import cross_val_score

In [ ]:
data = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')

In [ ]:
kaggleQ_timeResponse = pd.DataFrame(data)

In [ ]:
#Start off by making the response time to finish
kaggleQ_timeResponse.dropna(axis=0, subset=['Time from Start to Finish (seconds)'], inplace=True)
timeResponseColumn = (kaggleQ_timeResponse['Time from Start to Finish (seconds)'][1:]).astype(int)


In [ ]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(data[1:], timeResponseColumn, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

In [ ]:
if len(numerical_cols) == 0:
    print("It\'s all categorical data!! So we will keep all the data")

In [ ]:
X_train_full.drop(['Time from Start to Finish (seconds)'], axis=1, inplace=True)
X_valid_full.drop(['Time from Start to Finish (seconds)'], axis=1, inplace=True)

In [ ]:
def get_score(n_estimators):
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Define model
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=0)

    # Bundle preprocessing and modeling code in a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', model)
                         ])

    # Preprocessing of training data, fit model 
    clf.fit(X_train_full, y_train.tolist())

    # Preprocessing of validation data, get predictions
    preds = clf.predict(X_valid_full)
    
    return mean_absolute_error(y_valid.tolist(), preds)

In [ ]:
results = {} # Your code here
for i in range(1, 9):
    results[i] = get_score(i*50)

In [ ]:
for i in results.keys():
    print('MAE:', results[i])

In [ ]:
sns.lineplot(data=(timeResponseColumn))

In [ ]:
tests = data[1:]

In [ ]:
country_time = tests.astype(str).groupby('Q3')['Time from Start to Finish (seconds)'].agg(','.join).to_frame()
country_time.rename(columns = {'Time from Start to Finish (seconds)':'Time'}, inplace = True) 

In [ ]:
def countryTimes(colname):
    timeList = []
    try:
        for i in country_time[colname]:
            tempTime = 0
            i = i.split(',')
            #print(i)
            for x in i:
                tempTime += int(x)
            timeList.append(round(tempTime/len(i)))
        return timeList
    except:
        print("Try a numerical column!")

In [ ]:
countries = country_time.index.tolist()
timeList = countryTimes("Time")

In [ ]:
sorted_ZIP = sorted(zip(countries, timeList), key = lambda t: t[1])
unzipped_list = list(zip(*sorted_ZIP))

In [ ]:
### Fastest 7 countries on average
sns.barplot(x = list(unzipped_list[0][:7]), y = list(unzipped_list[1][:7]), estimator=np.median,capsize=.2, palette="Blues_d").set_title("Fastest Seven Countries on Average")

In [ ]:
### Slowest 7 countries
sns.barplot(x = list(unzipped_list[0][-7:]), y = list(unzipped_list[1][-7:]), estimator=np.median,capsize=.2, palette="Blues_d").set_title("Slowest Seven Countries on Average")

In [ ]:
number_of_people_by_country = data[1:].groupby('Q3')['Q2'].count()
countries = data[1:]['Q3'].sort_values(ascending=True)
countries = countries.unique()

# Top 7 countries represented
top_7 = number_of_people_by_country.sort_values(ascending=False)[:7]
top_7_countries = [x[:10] for x in top_7.index.tolist()]
top_7_number_people = top_7.tolist()
sns.barplot(x = top_7_countries, y = top_7_number_people).set_title("Participant Breakdown in Top Seven Countries")

In [ ]:
test_criterion_1 = data[1:].groupby(['Q3', 'Q2', 'Q4']).Q4.count()

In [ ]:
## Disproportionate amount of representation
sns.barplot(x = ["Top 7 Countries", "Total Whom Participated"], y = [sum(top_7_number_people), sum(number_of_people_by_country)]).set_title("Number of Participants in Top Seven Countries")

In [ ]:
top_10_number_people = number_of_people_by_country.sort_values(ascending=False)[:10].tolist()
## Disproportionate amount of representation
sns.barplot(x = ["Top 10 Countries", "Total Whom Participated"], y = [sum(top_10_number_people), sum(number_of_people_by_country)]).set_title("Number of Participants in Top Ten Countries")

In [ ]:
# Bottom 7 countries represented
bottom_7 = number_of_people_by_country.sort_values(ascending=False)[-7:]
bottom_7_countries = [x[:10] for x in bottom_7.index.tolist()]
bottom_7_number_people = bottom_7.tolist()
sns.barplot(x = bottom_7_countries, y = bottom_7_number_people).set_title("Bottom Seven Countries Represented")

In [ ]:
###### Number of women respresented in the survey
people_count = tests.groupby('Q2').Q2.count()
men = people_count[0]
women = people_count[4]
y = [x*3000 for x in range(1,7)]
print(y)
print(men)
sns.barplot(x = ["women", "men"], y = [women, men]).set_title("Number of Women Versus Men Represented")

In [ ]:
country_by_education = tests.groupby(['Q3', 'Q4']).Q4.count()

In [ ]:
doctoral_Education = tests.groupby('Q3')['Q4'].apply(lambda x: x[x == 'Doctoral degree'].count())
masters_Education = tests.groupby('Q3')['Q4'].apply(lambda x: x[x == 'Master’s degree'].count())

In [ ]:
### Top 10 countries by number of advanced degrees
top_10_Education = (doctoral_Education + masters_Education).sort_values(ascending=False)[:10]
top_10_countries_Education = [x[:7] for x in top_10_Education.index.tolist()]
top_10_Education_Numbers = top_10_Education.tolist()
sns.barplot(x = top_10_countries_Education, y = top_10_Education_Numbers).set_title("Top 10 Educated Countries By Advanced Degrees")

In [ ]:
print(country_by_education)

In [ ]:
### Top 10 countries by number of doctoral degrees
top_10_Education = (doctoral_Education).sort_values(ascending=False)[:10]
top_10_countries_Education = [x[:7] for x in top_10_Education.index.tolist()]
top_10_Education_Numbers = top_10_Education.tolist()
sns.barplot(x = top_10_countries_Education, y = top_10_Education_Numbers).set_title("Top 10 Educated Countries By Doctoral Degree")

In [ ]:
### Top 10 countries by number of masters degrees

top_10_Education = (masters_Education).sort_values(ascending=False)[:10]
top_10_countries_Education = [x[:7] for x in top_10_Education.index.tolist()]
top_10_Education_Numbers = top_10_Education.tolist()
sns.barplot(x = top_10_countries_Education, y = top_10_Education_Numbers).set_title("Top 10 Educated Countries By Master's Degrees")

In [ ]:
gender_doctoral_Education = tests.groupby('Q2')['Q4'].apply(lambda x: x[x == 'Doctoral degree'].count())
gender_masters_Education = tests.groupby('Q2')['Q4'].apply(lambda x: x[x == 'Master’s degree'].count())

In [ ]:
sns.barplot(x = ["women", "men"], y = [gender_masters_Education[4], gender_masters_Education[0]]).set_title("Gender Master\'s Degree Disparity")

In [ ]:
sns.barplot(x = ["women", "men"], y = [gender_doctoral_Education[4], gender_doctoral_Education[0]]).set_title("Gender Doctoral Degree Disparity")

In [ ]:

tests.to_csv('usedDF.csv', index=False)